## Цели исследования

проанализировать базу данных и  ответить на поставленные вопросы
В Базе данных информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

##  Установка подключения к базе

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных


connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [2]:
engine = create_engine(connection_string) 

## Задача: Исследуйте таблицы — выведите первые строки

In [3]:
query = '''
SELECT *
FROM books
limit (5  )

'''

In [4]:
books_head  = pd.read_sql_query(query, con=engine) 
books_head

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
query = '''
SELECT *
FROM authors
limit (5  )

'''

In [6]:
authors_head  = pd.read_sql_query(query, con=engine) 
authors_head

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
query = '''
SELECT *
FROM publishers
limit (5  )

'''

In [8]:
publishers_head  = pd.read_sql_query(query, con=engine) 
publishers_head

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [9]:
query = '''
SELECT *
FROM ratings
limit (5  )

'''

In [10]:
ratings_head  = pd.read_sql_query(query, con=engine) 
ratings_head

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [11]:
query = '''
SELECT *
FROM reviews
limit (5  )

'''

In [12]:
reviews_head  = pd.read_sql_query(query, con=engine) 
ratings_head

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Вывод: Посмотиели на структуру данных, но лучше всего она представлена на ER - диаграмме

## Задача 1 

Посчитайте, сколько книг вышло после 1 января 2000 года;

In [13]:
query = '''
SELECT COUNT (book_id)
FROM books
WHERE publication_date > '2000-01-01'

'''

In [14]:
task_1  = pd.read_sql_query(query, con=engine) 
task_1


,count
0,819


Вывод: после 1 января 2000 года вышло 819 книг.
всего в базе 1000 книги. То есть большая часть книг вышла после 2000 года

## Задача 2

Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [15]:
query = '''
WITH temple1 AS
(SELECT book_id,
COUNT(review_id) as reviews_count
FROM reviews
GROUP BY book_id),


temple2 AS
(SELECT book_id,
ROUND (AVG(rating),2) as avg_rating
FROM ratings
GROUP BY book_id)

SELECT b.title,
t1.reviews_count,
t2.avg_rating
FROM books AS b
LEFT JOIN temple1 AS t1 ON b.book_id = t1.book_id
LEFT JOIN temple2 AS t2 ON b.book_id = t2.book_id

'''

In [16]:
task_2  = pd.read_sql_query(query, con=engine) 
task_2


,title,reviews_count,avg_rating
0,The Body in the Library (Miss Marple #3),2.0,4.50
1,Galápagos,2.0,4.50
2,A Tree Grows in Brooklyn,5.0,4.25
3,Undaunted Courage: The Pioneering First Missio...,2.0,4.00
4,The Prophet,4.0,4.29
...,...,...,...
995,The Cat in the Hat and Other Dr. Seuss Favorites,NaN,5.00
996,Anne Rice's The Vampire Lestat: A Graphic Novel,NaN,3.67
997,Essential Tales and Poems,NaN,4.00
998,Leonardo's Notebooks,NaN,4.00


Вывод: Не у всех книг есть обзоры) 

## Задача 3

Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [17]:
query = '''
SELECT 
p.publisher,
COUNT(b.book_id)


FROM books as b
LEFT JOIN publishers as p ON b.publisher_id=p.publisher_id
WHERE b.num_pages>50
GROUP BY p.publisher
ORDER BY COUNT(b.book_id) DESC
LIMIT (1)
'''

In [18]:
task_3  = pd.read_sql_query(query, con=engine) 
task_3

,publisher,count
0,Penguin Books,42


Вывод : Издательство Penguin Books выпустило больше всех книг (более 50 страниц). 

## Задача 4

Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [19]:
query = '''
WITH tem1 as 
(SELECT
book_id,
COUNT(rating) as count_rating,
ROUND (AVG (rating),2)  as avg_rating
FROM ratings
GROUP BY book_id
ORDER BY 2 DESC),

tem2 AS

(SELECT
t1.book_id,
r.rating,
a.author

FROM tem1 as t1
LEFT JOIN ratings as r ON t1.book_id = r.book_id
LEFT JOIN  books as b ON t1.book_id = b.book_id
LEFT JOIN authors as a ON b.author_id = a.author_id

WHERE t1.count_rating>=50)


SELECT author,
AVG(rating)
FROM tem2
GROUP BY 1 
ORDER BY 2 DESC
LIMIT (1)

'''

In [20]:
task_4  = pd.read_sql_query(query, con=engine) 
task_4

,author,avg
0,J.K. Rowling/Mary GrandPré,4.287097


Вывод: Автор бестселлеров про Гарри Поттера (ныне отмененная) Джоан Роллинг имеет самые лучшие рейтинги

In [61]:
query = '''
WITH tem1 as 

(SELECT
book_id,
COUNT(rating) as count_rating,
AVG (rating)  as avg_rating
FROM ratings
GROUP BY book_id
ORDER BY 2 DESC
LIMIT 19), 

tem2 as 


(SELECT
t1.book_id,
t1.avg_rating,
a.author

FROM tem1 as t1

LEFT JOIN  books as b ON t1.book_id = b.book_id
LEFT JOIN authors as a ON b.author_id = a.author_id) 

SELECT author,
AVG(avg_rating)
FROM tem2
GROUP BY 1 
ORDER BY 2 DESC
LIMIT (1)




'''

In [62]:
task_6  = pd.read_sql_query(query, con=engine) 
task_6

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


## Задача 5

Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [21]:
query = '''
SELECT AVG (temple.total_review)

FROM 
(SELECT 
COUNT(review_id) as total_review

FROM reviews
WHERE username IN (
SELECT 
username
FROM ratings
GROUP BY username
HAVING COUNT (rating)>50)

GROUP BY username) as temple



'''

In [22]:
task_5  = pd.read_sql_query(query, con=engine) 
task_5

,avg
0,24.333333


Вывод: топовые пользователи в среднем пишут по 24 обзора. 


<div class="alert alert-info"> <b>Комментарий студента:</b>  
    
    Выводы:
  В результате нашего мини исследования получили такие результаты: 
    
    1. после 1 января 2000 года вышло 819 книг. всего в базе 1000 книги. То есть большая часть книг вышла после 2000 года
    2. Посчитала среднее количество обзоров  и среднюю оценку для каждой книги. Вывод -  не у всех книг есть обзоры 
    3. Определили издательство, выпустившее наибольшее число книг толще 50 страниц  - это издательство Penguin Books 
    4. Нашли автора  с самой высокой средней оценкой книг. - бестселлеров 9то есть тех книг, где больше 50 оценок) Это J.K. Rowling
    5.Посчитала среднее количество обзоров от пользователей, которые поставили больше 50 оценок. Среднее количество обзоров от топовых пользователей - 24 обзора) 